<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/stock_data_collect_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Data Collecting and Web Crawling

Adapted from [最強AI投資分析(旗標)](https://www.tenlong.com.tw/products/9789863127727)

## 3-2 資料爬蟲

## 證交所資料擷取
1. 進入證交所網址：https://www.twse.com.tw/zh/index.html
2. 使用開發者模式取得請求資料網址

### 1️⃣ 匯入套件

In [ ]:
import requests
import pandas as pd
import datetime as dt # 時間套件
from dateutil.relativedelta import relativedelta

### 2️⃣ 取得個股日成交資訊

In [ ]:
# 輸入股票代號
stock_id = '2330'
# 當日時間
date = dt.date.today().strftime("%Y%m%d")
# 取得證交所網站資料
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')
json_data = stock_data.json()
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
df.tail()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
8,114/01/14,"31,280,235","33,915,048,410","1,085.00","1,090.00","1,075.00","1,090.00",+15.00,"32,218"
9,114/01/15,"37,966,582","40,677,529,813","1,085.00","1,085.00","1,065.00","1,065.00",-25.00,"74,906"
10,114/01/16,"49,872,673","54,934,128,041","1,095.00","1,115.00","1,090.00","1,105.00",+40.00,"54,389"
11,114/01/17,"60,857,371","67,663,576,840","1,125.00","1,125.00","1,095.00","1,120.00",+15.00,"48,829"
12,114/01/20,"32,062,171","36,076,248,367","1,125.00","1,135.00","1,120.00","1,120.00",0.00,"37,780"


### 3️⃣ 取得連續月份資料
以個股本益比為例

In [ ]:
# 設定抓取幾個月資料
month_num=3
date_now = dt.datetime.now()

# 建立日期串列
date_list = [(date_now - relativedelta(months=i)).replace(day=1).\
             strftime('%Y%m%d') for i in range(month_num)]

date_list.reverse()
all_df = pd.DataFrame()

# 使用迴圈抓取連續月份資料
for date in date_list:
  url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
      BWIBBU?date={date}&stockNo={stock_id}'
  try:
    json_data = requests.get(url).json()
    df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
    all_df = pd.concat([all_df, df], ignore_index=True)
  except Exception as e:
    print(f"無法取得{date}的資料, 可能資料量不足.")

all_df.head()

,日期,殖利率(%),股利年度,本益比,股價淨值比,財報年/季
0,113年11月01日,1.27,112,28.79,7.01,113/2
1,113年11月04日,1.25,112,29.21,7.11,113/2
2,113年11月05日,1.24,112,29.49,7.18,113/2
3,113年11月06日,1.23,112,29.78,7.25,113/2
4,113年11月07日,1.22,112,29.92,7.28,113/2


## 用 BeautifulSoup4 取得 Yahoo 股市資料

###4️⃣ 匯入套件

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import time

###  5️⃣ 取得當日股價

In [ ]:
def yahoo_stock(stock_id):
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'
    # 使用 requests 取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用 Beautiful Soup 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')
    # 使用 find 與 find_all 定位元素
    time_element = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('time')
    table_soups = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('ul')\
                                   .find_all('li')
    fields = []
    datas = []
    for table_soup in table_soups:
        table_datas = table_soup.find_all('span')
        for num,table_data in enumerate(table_datas):
            if table_data.text =='':
                continue
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)
    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)
    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_id)
    # 回傳 DataFrame
    return df

yahoo_stock(stock_id)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2025/01/21 11:19,2330,"1,115","1,115","1,120","1,110","1,115",93.74,"1,120",0.45%,5.00,"8,402","29,797",0.89%


### 6️⃣ 取得季報表資訊


In [ ]:
# 函式可用於奇摩財報
def url_find(url):
    words = url.split('/')
    k = words[-1]
    # 使用requests取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用Beautiful Soup解析HTML內容
    soup = BeautifulSoup(html, 'html.parser')
    # 找到表格的表頭
    table_soup = soup.find('section', {'id': 'qsp-{}-table'.format(k)})
    table_fields=table_soup.find('div', class_='table-header')
    table_fields_lines = list(table_fields.stripped_strings)
    # 找到數據
    data_rows = table_soup.find_all('li' ,class_='List(n)')
    # 解析資料行內容
    data = []
    for row in data_rows:
        row_data = list(row.stripped_strings)
        data.append(row_data)
    # 建立 DataFrame
    df = pd.DataFrame(data, columns=table_fields_lines)
    return df

# 抓損益表
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
# 抓資產負債表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/balance-sheet'
# 抓現金流量表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/cash-flow-statement'

# 抓取季報表資料
df = url_find(url).transpose()

# 資料處理
df.columns = df.iloc[0]
df = df[1:]
df.insert(0,'年度/季別',df.index)
df.columns.name = None
df.reset_index(drop=True, inplace=True)

df.tail()

,年度/季別,營業收入,營業毛利,營業費用,營業利益,稅後淨利
15,2020 Q4,"361,533,057","195,188,826","38,321,439","157,120,174","142,824,524"
16,2020 Q3,"356,426,204","190,480,670","40,891,355","150,047,349","137,378,151"
17,2020 Q2,"310,698,367","164,692,825","33,520,709","131,094,538","120,892,514"
18,2020 Q1,"310,597,183","160,784,181","32,323,046","128,521,637","117,062,893"
19,2019 Q4,"317,237,065","159,240,985","34,942,621","124,243,722","116,078,194"


##使用 selenium 做新聞爬蟲

### 7️⃣ 用 requests 取得股票新聞

In [ ]:
# 股票代號
stock_id = "2330"
# 預設表格數據和欄位
field=['股號','日期','標題','內容']
data=[]
# 取得 Json 格式資料
json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/'
                f'v1/news/keyword?q={stock_id}&limit=20&page=1').json()
# 依照格式擷取資料
items=json_data['data']['items']
for item in items:
    # 網址、標題和日期
    news_id = item["newsId"]
    title = item["title"]
    publish_at = item["publishAt"]
    # 使用 UTC 時間格式
    utc_time = datetime.utcfromtimestamp(publish_at)
    formatted_date = utc_time.strftime('%Y-%m-%d')
    # 前往網址擷取內容
    url = requests.get(f'https://news.cnyes.com/'
                       f'news/id/{news_id}').content
    soup = BeautifulSoup(url, 'html.parser')
    p_elements = soup.find_all('p')
    # 提取段落内容
    p=''
    for paragraph in p_elements[4:]:
        p+=paragraph.get_text()
    data.append([stock_id, formatted_date ,title,p])
# 建立表格
df = pd.DataFrame(data,columns=field)
df

,股號,日期,標題,內容
0,2330,2025-01-20,台積電:本公司代重要子公司TSMC North America公告股東常會重要決議,3.重要決議事項二、章程修訂:不適用。4.重要決議事項三、營業報告書及財務報表:不適用。5....
1,2330,2025-01-20,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:114/1/20~114/1/203.交易數量、每單位價格及交易總金額:060...
2,2330,2025-01-18,鉅亨速報 - Factset 最新調查：台積電(<mark>2330</mark>-TW)目...,資料來源：Factset，數據僅供參考，不作為投資建議。上一篇下一篇張國華妻信託2.67萬張...
3,2330,2025-01-17,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:114/1/17~114/1/173.交易數量、每單位價格及交易總金額:616...
4,2330,2025-01-17,"《異動股》中國生物製藥(01177)升近3%,現報2.89元",基本資料 ==== 股份狀態:主動買沽...
5,2330,2025-01-17,鉅亨速報 - Factset 最新調查：台積電(<mark>2330</mark>-TW)E...,歷史獲利表現詳細資訊請看台股內頁：https://www.cnyes.com/twstock...
6,2330,2025-01-16,台積電:台積公司2024年第四季每股盈餘新台幣14.45元,3.與公司關係(請輸入本公司或子公司):本公司4.相互持股比例:不適用5.發生緣由:不適用6...
7,2330,2025-01-15,【量大強漲股整理】台積電明天法說會，供應鏈如何看待，行情如何解讀?,台股方面，今日適逢台指期結算影響下，加上近期輝達傳出多項利空消息，以及台積電法說會前的觀望，...
8,2330,2025-01-15,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:113/12/26~114/1/153.交易數量、每單位價格及交易總金額:61...
9,2330,2025-01-14,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,2.交易日期:114/1/14~114/1/143.交易數量、每單位價格及交易總金額:617...


### 8️⃣  安裝及匯入套件

In [ ]:
!pip install selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 不顯示瀏覽器
chrome_options.add_argument('--no-sandbox')# 以最高權限運行

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 20.1 MB/s eta 0:00:00


### 9️⃣ 使用 Selenium 取得股票新聞

In [ ]:
# 透過 options 設定 driver
driver = webdriver.Chrome(options=chrome_options)
data2=[] # 表格數據
# 目標網址
url = f"https://www.cnyes.com/search/news?keyword={stock_id}"
driver.get(url)

# 模擬滑動滑鼠滾輪的行為，用於加載更多內容
scroll_pause_time = 2  # 等待時間
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script(
        "window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script(
        "return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

elements = driver.find_elements("xpath",'//*[@id="_SearchAll"]/section/div/a')

# 擷取網址和標題
for element in elements:
    link = element.get_attribute("href")
    title = element.text
    title=title.split('\n')
    data2.append([stock_id, title[1] ,title[0],link])
# 關閉瀏覽器
driver.quit()
for link in data2:
  # 使用 requests 前往網址擷取新聞內容
  link_a = requests.get(link[3]).content
  link_b = BeautifulSoup(link_a,'html.parser')
  p_elements = link_b.find('article')
  # 取得段落内容
  try:
    link[3] = p_elements.get_text()
  except:
    link[3] = '無內容'
# 建立表格
df = pd.DataFrame(data2,columns=field)
df.tail()

,股號,日期,標題,內容
728,2330,2023/02/03,台積電先行 引爆比價效應,雜誌台積電先行 引爆比價效應理財周刊 2023-02-03 08:06‌\n文．高適\n...
729,2330,2023/02/02,謝金河：元月效應有吉兆,雜誌謝金河：元月效應有吉兆先探投資週刊 2023-02-02 14:13‌\n【文／謝金...
730,2330,2023/01/31,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,公告台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券鉅亨網新...
731,2330,2023/01/30,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,公告台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券鉅亨網新...
732,2330,2023/01/30,盤中速報 - 台積電(2330)股價大漲至541.0元，漲幅達7.55%,鉅亨速報盤中速報 - 台積電(2330)股價大漲至541.0元，漲幅達7.55%鉅亨網新聞...


```
#把資料存成 csv 可以用以下這段
df.to_csv('/content/news_data.csv' )
```

#3.3 用 Python 套件輕鬆取得股市資料

##使用 yfinance 下載股市資料

### 🔟  安裝及匯入套件

In [ ]:
!pip install yfinance==0.2.38
import yfinance as yf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.51
    Uninstalling yfinance-0.2.51:
      Successfully uninstalled yfinance-0.2.51


### 1️⃣1️⃣  設定股票代碼和起止時間

In [ ]:
# 指定要下載的股票代碼, 上市為 .TW;上櫃為 .TWO
stock_id = '2330.TW'
# 設定開始與結束時間
end = dt.date.today()
start = end - dt.timedelta(days=360)

### 1️⃣2️⃣ 取得每日股價 (開高低收) 資料

In [ ]:
stock_data = yf.download(stock_id, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2025-01-14,1085.0,1090.0,1075.0,1090.0,1090.0,28695247
2025-01-15,1085.0,1085.0,1065.0,1065.0,1065.0,35489482
2025-01-16,1095.0,1115.0,1090.0,1105.0,1105.0,47542962
2025-01-17,1125.0,1125.0,1095.0,1120.0,1120.0,59214050
2025-01-20,1125.0,1135.0,1120.0,1120.0,1120.0,29834528




```
# 依照資料期間下載
stock_data = yf.download(stock_id, period="3mo")

# 下載不同時間頻率的資料 (1分K)
stock_data = yf.download(stock_id, interval="1m")
```



### 1️⃣3️⃣ 取得多檔股票的資料

In [ ]:
stocks = [stock_id, '2303.TW', '2454.TW'] #分別為台積電、聯電和聯發科
stock_data = yf.download(stocks, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  3 of 3 completed


Price       Adj Close                      Close                    High  \
Ticker        2303.TW 2330.TW 2454.TW    2303.TW 2330.TW 2454.TW 2303.TW   
Date                                                                       
2025-01-14  42.250000  1090.0  1400.0  42.250000  1090.0  1400.0   42.25   
2025-01-15  41.450001  1065.0  1420.0  41.450001  1065.0  1420.0   42.50   
2025-01-16  41.549999  1105.0  1460.0  41.549999  1105.0  1460.0   42.25   
2025-01-17  41.200001  1120.0  1430.0  41.200001  1120.0  1430.0   41.75   
2025-01-20  42.750000  1120.0  1445.0  42.750000  1120.0  1445.0   42.75   

Price                             Low                       Open          \
Ticker     2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW 2330.TW   
Date                                                                       
2025-01-14  1090.0  1415.0  41.049999  1075.0  1380.0  41.200001  1085.0   
2025-01-15  1085.0  1445.0  41.450001  1065.0  1390.0  42.049999  1085.0   
2025-01-16  1115.0  1470.0  41.549999  1090.0  1430.0  41.900002  1095.0   
2025-01-17  1125.0  1455.0  41.200001  1095.0  1420.0  41.400002  1125.0   
2025-01-20  1135.0  1455.0  41.500000  1120.0  1425.0  41.549999  1125.0   

Price                 Volume                     
Ticker     2454.TW   2303.TW   2330.TW  2454.TW  
Date                                             
2025-01-14  1385.0  37074886  28695247  5279672  
2025-01-15  1390.0  35977056  35489482  6417175  
2025-01-16  1450.0  37293479  47542962  7314623  
2025-01-17  1450.0  44131855  59214050  5488719  
2025-01-20  1425.0  47283878  29834528  3956489

### 1️⃣4️⃣ 取得公司基本資料


In [ ]:
stock = yf.Ticker(stock_id)
stock.info # 為字典型態

{'address1': 'Hsinchu Science Park',
 'address2': 'No. 8, Li-Hsin Road 6',
 'city': 'Hsinchu City',
 'zip': '300096',
 'country': 'Taiwan',
 'phone': '886 3 563 6688',
 'fax': '886 3 563 7000',
 'website': 'https://www.tsmc.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support se

### 1️⃣5️⃣  取得損益表

In [ ]:
financials = stock.financials
financials.tail()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Other Gand A,60872800000.0,53524800000.0,36929600000.0,28457600000.0
Gross Profit,1175110600000.0,1348354800000.0,819537300000.0,711130100000.0
Cost Of Revenue,986625200000.0,915536500000.0,767877700000.0,628124700000.0
Total Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0
Operating Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0


In [ ]:
bs = stock.balance_sheet
bs

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Share Issued,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Total Debt,956257900000.0,888174400000.0,753631900000.0,367792300000.0
Tangible Book Value,3406755700000.0,2877020500000.0,2122438100000.0,1809043200000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,1714803200000.0,1586500100000.0,1204913700000.0,799893200000.0
Other Short Term Investments,249375400000.0,243686000000.0,139923500000.0,139722600000.0
Cash And Cash Equivalents,1465427800000.0,1342814100000.0,1064990200000.0,660170600000.0
Cash Equivalents,12326200000.0,13522700000.0,6182100000.0,6590000000.0


## 使用 FinMind 下載股市資料

### 1️⃣7️⃣  安裝及匯入套件

In [ ]:
!pip install FinMind
from FinMind.data import DataLoader
import getpass

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/151.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 10.8 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24860 sha256=db3189e9fbf9938d226036289786587da2724e0dcbd0a751d3fd9f9fa84544a3
  Stored in directory: /root/.cache/pip/wheels/33/fc/77/8c7e03a6570d67208d06005e9358a62fabc17eb963e7baf75b
Successfully built ta


### 1️⃣8️⃣  輸入 FinMind API 和帳號密碼

In [ ]:
token = getpass.getpass("請輸入 FinMind 金鑰：")

請輸入 FinMind 金鑰：··········


### 1️⃣9️⃣  建立 FinMind 資料庫物件和登入 FinMind

In [ ]:
api = DataLoader()
api.login_by_token(api_token=token)

### 2️⃣0️⃣ 取得股價資料

In [ ]:
# 股票代號
stock_id = '2330'
# 資料期間
end = dt.date.today()
start = end - dt.timedelta(days=360)

stock_data =  api.taiwan_stock_daily(
    stock_id=stock_id,
    start_date=start,
    end_date=end)

stock_data.tail()

2025-01-21 03:38:04.533 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 2330


,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
231,2025-01-14,2330,31280235,33915048410,1085.0,1090.0,1075.0,1090.0,15.0,32218
232,2025-01-15,2330,37966582,40677529813,1085.0,1085.0,1065.0,1065.0,-25.0,74906
233,2025-01-16,2330,49872673,54934128041,1095.0,1115.0,1090.0,1105.0,40.0,54389
234,2025-01-17,2330,60857371,67663576840,1125.0,1125.0,1095.0,1120.0,15.0,48829
235,2025-01-20,2330,32062171,36076248367,1125.0,1135.0,1120.0,1120.0,0.0,37780


### 2️⃣1️⃣ 取得損益表資料

In [ ]:
financial_data = api.taiwan_stock_financial_statement(
    stock_id=stock_id,
    start_date=str(start),)

financial_data.tail()

2025-01-21 03:38:39.350 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockFinancialStatements, data_id: 2330


,date,stock_id,type,value,origin_name
43,2024-09-30,2330,TotalNonoperatingIncomeAndExpense,2.342056e+10,營業外收入及支出
44,2024-09-30,2330,OtherComprehensiveIncome,-2.105628e+10,其他綜合損益（淨額）
45,2024-09-30,2330,EquityAttributableToOwnersOfParent,3.021003e+11,綜合損益總額歸屬於母公司業主
46,2024-09-30,2330,NoncontrollingInterests,1.923569e+09,綜合損益總額歸屬於非控制權益
47,2024-09-30,2330,GrossProfit,4.393457e+11,營業毛利（毛損）


### 2️⃣2️⃣ 取得法人買賣資料

In [ ]:
investors_data = api.taiwan_stock_institutional_investors(
    stock_id=stock_id,
    start_date=str(start),)
investors_data.tail()

2025-01-21 03:38:46.921 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInstitutionalInvestorsBuySell, data_id: 2330


,date,stock_id,buy,name,sell
1175,2025-01-20,2330,0,Foreign_Dealer_Self,0
1176,2025-01-20,2330,200100,Dealer_self,99500
1177,2025-01-20,2330,154400,Dealer_Hedging,54643
1178,2025-01-20,2330,17833303,Foreign_Investor,13672835
1179,2025-01-20,2330,52500,Investment_Trust,468164


## 使用 FinLab 下載股市資料
注意！FinLab 目前需付費才能取得最新資料

### 2️⃣3️⃣ 安裝及匯入套件

In [ ]:
!pip install finlab
import finlab
from finlab import data

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00


### 2️⃣4️⃣ 登入 FinLab

In [ ]:
token = getpass.getpass("請輸入FinLab金鑰：")
finlab.login(token)

請輸入FinLab金鑰：··········
輸入成功!


### 2️⃣5️⃣ 取得收盤價

In [ ]:
close = data.get('price:收盤價')
close.tail()

Due to your status as a free user, the most recent data has been shortened or limited.
Daily usage: 25.1 / 500 MB - price:收盤價


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2025-01-14,NaN,195.60,75.05,193.45,102.90,NaN,27.99,36.36,143.15,NaN,...,20.40,41.00,20.80,25.55,13.9,54.7,28.20,154.0,25.20,13.80
2025-01-15,NaN,194.10,74.65,192.50,101.70,NaN,28.00,36.12,141.65,NaN,...,20.25,40.80,20.50,25.45,14.1,55.4,28.20,151.0,NaN,13.95
2025-01-16,NaN,198.05,75.55,196.70,104.65,NaN,28.20,36.39,145.35,NaN,...,20.45,41.55,20.65,25.70,14.0,55.7,28.50,155.0,NaN,13.80
2025-01-17,NaN,196.30,75.70,197.50,107.00,NaN,28.17,35.35,145.25,NaN,...,20.45,41.75,20.70,25.35,14.0,56.0,28.25,154.0,NaN,13.90
2025-01-20,NaN,198.30,76.45,199.10,106.20,NaN,28.10,35.52,146.80,NaN,...,20.50,41.95,21.30,25.30,14.0,55.5,28.10,156.0,25.35,13.95


### 2️⃣6️⃣ 選擇產業

In [ ]:
data.set_universe(market='TSE', category='半導體')
close = data.get('price:收盤價')
close.tail()

Daily usage: 25.2 / 500 MB - security_categories


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2025-01-14,18.15,42.25,31.95,1090.0,18.25,46.00,26.15,29.50,13.55,88.0,...,318.0,208.0,122.5,223.5,13.00,27.75,29.90,39.85,82.0,43.10
2025-01-15,18.20,41.45,32.00,1065.0,18.00,45.65,26.30,29.55,13.65,86.2,...,323.0,208.0,120.0,223.0,13.10,28.00,29.90,39.80,80.6,43.25
2025-01-16,18.25,41.55,32.60,1105.0,18.45,46.45,27.05,30.25,14.00,88.0,...,335.0,208.5,128.0,225.0,13.25,28.00,30.00,39.20,81.1,44.30
2025-01-17,18.20,41.20,32.45,1120.0,19.00,47.00,26.70,30.10,14.05,89.0,...,323.5,207.0,121.5,228.0,13.25,28.70,30.30,39.85,80.6,44.15
2025-01-20,18.15,42.75,33.15,1120.0,19.50,47.65,27.05,30.50,14.10,90.7,...,324.0,209.5,125.5,227.5,13.30,28.75,30.25,39.85,81.1,44.90


### 2️⃣7️⃣ 取得財報資料

In [ ]:
df = data.get('financial_statement:每股盈餘')
df.tail()

Daily usage: 26.5 / 500 MB - financial_statement:每股盈餘


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2019-Q3,0.02,0.25,0.45,3.90,1.01,0.66,0.50,-1.10,0.15,0.69,...,NaN,3.22,0.83,2.50,0.41,0.80,0.81,NaN,0.22,1.34
2019-Q4,-0.08,0.32,0.08,4.48,0.41,0.62,0.38,-0.89,-0.05,0.31,...,NaN,2.54,0.44,2.19,0.32,0.69,0.72,NaN,0.11,0.99
2020-Q1,-0.09,0.19,0.02,4.51,0.67,-1.09,0.32,-0.69,-0.02,0.27,...,NaN,2.43,0.28,2.73,-0.27,0.88,0.98,NaN,0.29,1.53
2020-Q2,0.01,0.55,-0.50,4.66,0.72,0.66,0.19,-0.91,0.18,0.38,...,NaN,2.43,0.31,2.73,0.18,0.78,0.75,NaN,0.57,0.57
2020-Q3,0.21,0.75,0.02,5.30,0.88,1.27,0.55,2.00,0.08,0.51,...,NaN,2.63,0.11,3.55,0.42,0.80,0.58,NaN,0.68,0.24


### 2️⃣8️⃣  取得法人資料

In [ ]:
df = data.get('institutional_investors_trading_summary:投信買賣超股數')
df.tail()

Daily usage: 33.5 / 500 MB - institutional_investors_trading_summary:投信買賣超股數


symbol,2302,2303,2329,2330,2337,2338,2340,2342,2344,2351,...,6937,8016,8028,8081,8110,8131,8150,8162,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2024-09-10,0.0,3333120.0,0.0,414999.0,9000.0,0.0,0.0,0.0,415826.0,150000.0,...,20000.0,-72696.0,0.0,-72984.0,0.0,0.0,14133.0,0.0,3864.0,5803.0
2024-09-11,0.0,8646605.0,1000.0,658261.0,6000.0,0.0,0.0,0.0,26000.0,0.0,...,4000.0,-17936.0,5000.0,-76443.0,0.0,0.0,49469.0,0.0,13524.0,10449.0
2024-09-12,0.0,2762900.0,-1000.0,70628.0,0.0,0.0,0.0,0.0,4826.0,0.0,...,0.0,-65554.0,0.0,-76459.0,0.0,0.0,20498.0,NaN,9659.0,5802.0
2024-09-13,0.0,6616943.0,0.0,49470.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2000.0,-5122.0,45000.0,-78475.0,0.0,0.0,17746.0,0.0,5797.0,12766.0
2024-09-16,0.0,3871137.0,0.0,386315.0,19000.0,0.0,0.0,0.0,22913.0,84000.0,...,0.0,-12279.0,0.0,-75475.0,0.0,0.0,10827.0,NaN,5797.0,5792.0


#3.4 SQL資料庫

### 2️⃣9️⃣ 匯入套件及連線資料庫

In [ ]:
import sqlite3
conn = sqlite3.connect('stock.db')

### 3️⃣0️⃣設定資料庫結構

In [ ]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS 日頻資料 (
    sno INTEGER PRIMARY KEY AUTOINCREMENT,
    Stock_Id TEXT,
    Date DATE,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Adj_Close FLOAT,
    Volume INTEGER
);
''')
conn.commit()

### 3️⃣1️⃣ 傳入資料到資料庫

In [ ]:
df = yf.download('2330.TW',start='2023-08-01')
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
df.insert(0,'Stock_id','2330')

df.to_sql('日頻資料',conn,if_exists='append',index=False)

[*********************100%%**********************]  1 of 1 completed


361

### 3️⃣2️⃣ 查詢表格資料

In [ ]:
def table_info(table_name):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns]
    print(f"資料庫表 '{table_name}' 的欄位名稱：", column_names)
    all_data = conn.execute(f'SELECT * FROM {table_name}')
    for i in all_data.fetchall():
        print(i)

# 查詢表格資料
table_info("日頻資料")

資料庫表 '日頻資料' 的欄位名稱： ['sno', 'Stock_Id', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
(1, '2330', '2023-08-01', 565.0, 568.0, 564.0, 567.0, 551.5789184570312, 16259643)
(2, '2330', '2023-08-02', 567.0, 569.0, 558.0, 561.0, 545.7421264648438, 25583234)
(3, '2330', '2023-08-04', 556.0, 560.0, 552.0, 554.0, 538.9324340820312, 26279173)
(4, '2330', '2023-08-07', 558.0, 561.0, 556.0, 558.0, 542.82373046875, 14369551)
(5, '2330', '2023-08-08', 558.0, 558.0, 551.0, 552.0, 536.98681640625, 19010690)
(6, '2330', '2023-08-09', 550.0, 557.0, 550.0, 554.0, 538.9324340820312, 14389090)
(7, '2330', '2023-08-10', 552.0, 554.0, 550.0, 551.0, 536.0140380859375, 17892615)
(8, '2330', '2023-08-11', 556.0, 558.0, 546.0, 546.0, 531.1500244140625, 17608427)
(9, '2330', '2023-08-14', 540.0, 543.0, 537.0, 541.0, 526.2860107421875, 20939030)
(10, '2330', '2023-08-15', 543.0, 545.0, 540.0, 542.0, 527.2588500976562, 14066307)
(11, '2330', '2023-08-16', 541.0, 543.0, 536.0, 542.0, 527.258850097656

### 3️⃣3️⃣ 新增資料

In [ ]:
def insert_data(stock_id, start):
  # 下載資料
  df = yf.download(f'{stock_id}.TW',start=start)
  df = df.reset_index()
  df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
  df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
  df.insert(0,'Stock_Id',stock_id)

  # 新增資料表
  df.to_sql('日頻資料',conn,if_exists='append',index=False)

# 新增 2317 資料
insert_data(stock_id=2317, start='2023-08-01')

[*********************100%%**********************]  1 of 1 completed


### 3️⃣4️⃣ 從資料庫取得資料

In [ ]:
query = ("SELECT Stock_id, Date, Close "
         "FROM 日頻資料 "
         "WHERE Date < '2023-08-15' AND Stock_id = '2317'")
df = pd.read_sql(query, conn, parse_dates=['Date'])
df.tail()

,Stock_Id,Date,Close
4,2317,2023-08-08,110.5
5,2317,2023-08-09,110.5
6,2317,2023-08-10,110.0
7,2317,2023-08-11,108.5
8,2317,2023-08-14,110.0


### 3️⃣5️⃣ 修改並關閉資料庫

In [ ]:
conn.commit()
conn.close()